In [1]:
%%capture
import stata_setup, os
if os.name == 'nt':
    stata_setup.config('C:/Program Files/Stata17/','mp')
else:
    stata_setup.config('/usr/local/stata17','mp')

## Preparing the data

In [2]:
%%stata -qui

use "../data/data", clear
gen prop_women_100 = prop_women*100 
rename log_flesch_kincaid_grade_level FKG
quietly tabulate year, generate(y_)
quietly tabulate cluster, generate(c_)

local journals  ecm jpe qje res  //AER based category

local jel_imp a_imp b_imp c_imp  e_imp f_imp g_imp h_imp i_imp j_imp k_imp /// 
		l_imp m_imp n_imp o_imp p_imp q_imp r_imp y_imp z_imp // D JEL based case

## Post-Regularization Lasso

The Post-Regularization (Partialing-Out) Lasso is implemented by the ```poregress``` Stata command. It'll utilize a 'plugin' value for the $\lambda$s parameters by default, but they can also use standard cross-validation instead.

In [3]:
%%stata -qui
#delimit ;
poregress FKG  log_num_authors log_num_pages both_genders prop_women_100,
			controls(`journals' `jel_imp' y_2-y_20  c_2-c_215  jel_flag)
            vce(cluster cluster) rseed(42);
estimates store po_plugin;
poregress FKG  log_num_authors log_num_pages both_genders prop_women_100,
			controls(`journals' `jel_imp' y_2-y_20  c_2-c_215  jel_flag)
            vce(cluster cluster) selection(cv) rseed(42);
estimates store po_cv;
#delimit cr

In [4]:
%%stata
#delimit ;
lassocoef (po_plugin, for(FKG)) (po_cv, for(FKG))
          (po_plugin, for(prop_women_100)) (po_cv, for(prop_women_100));
#delimit cr


. #delimit ;
delimiter now ;
. lassocoef (po_plugin, for(FKG)) (po_cv, for(FKG))
>           (po_plugin, for(prop_women_100)) (po_cv, for(prop_women_100));

-------------------------------------------------------------------
             | po_plugin   po_cv       po_plugin         po_cv     
             |    FKG       FKG     prop_women_100  prop_women_100 
-------------+-----------------------------------------------------
     x    _4 |
        c_36 |     x    
        c_50 |     x    
       c_165 |     x    
       c_183 |     x    
        c_56 |                             x       
        c_91 |                             x       
       c_100 |                             x       
       c_174 |                             x       
       c_196 |                             x       
       c_imp |                                             x       
       i_imp |                                             x       
       k_imp |                                             

In [5]:
%%stata
estimates restore po_cv
poregress


. estimates restore po_cv
(results po_cv are active now)

. poregress

Partialing-out linear model          Number of obs                =      4,988
                                     Number of controls           =        257
                                     Number of selected controls  =         37
                                     Wald chi2(4)                 =      11.16
                                     Prob > chi2                  =     0.0248

                              (Std. err. adjusted for 215 clusters in cluster)
------------------------------------------------------------------------------
             |               Robust
         FKG | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
log_num_au~s |  -.0029956   .0051732    -0.58   0.563    -.0131349    .0071438
log_num_pa~s |   .0161292   .0055794     2.89   0.004     .0051939    .0270646
both_genders |  -.0021